In [ ]:
! pip install datasets

     |████████████████████████████████| 225kB 6.5MB/s 
     |████████████████████████████████| 245kB 8.6MB/s 
     |████████████████████████████████| 112kB 9.4MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 6.6MB/s 
     |████████████████████████████████| 901kB 35.0MB/s 
     |████████████████████████████████| 3.3MB 40.0MB/s 


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
import datasets
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_fname = "/content/drive/Shareddrives/CS109B-2021/final_project/data/subset/case_info.json"
# cases_df = pd.read_json(data_fname, lines=True, orient="records")[["id", "head_matter", "opinion_text"]]
# cases_df["text"] = cases_df["head_matter"] + "\n" +  cases_df["opinion_text"]
# cases_df = cases_df[["id", "text"]]

In [ ]:
! head {data_fname}

{"id":1879972,"jurisdiction_id":34,"court_id":8808,"decision_date":463622400000,"head_matter":"Curtis HOWARD and Billy HOWARD v. STATE of Arkansas\nCR 84-7\n674-S.W.2d 936\nSupreme Court of Arkansas\nOpinion delivered September 10, 1984\nJames M. Simpson, for appellant Curtis Howard.\nCynthia L. Fearno, for appellant Billy Howard.\nSteve Clark, Att\u2019y Gen., by: Velda West Vanderbilt, Asst. Att\u2019y Gen., for appellee.","opinion_text":"George Rose Smith, Justice.\nThe two appellants, Curtis and Billy Howard, brothers, were charged with and convicted of aggravated robbery and theft of property. Curtis, as an habitual offender, was sentenced to consecutive terms of life and 30 years. Billy was sentenced to concurrent terms of 10 and 5 years. In separate briefs they argue that owing to various gaps in the State\u2019s proof it is insufficient to sustain the convictions. Viewing the evidence most favorably to the appellee, we find it amply sufficient. In narrating the events as the ju

In [ ]:
model_name = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name, from_pt=True)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

In [ ]:
# Check # of CPUS
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         148G   40G  108G  27% /
tmpfs            64M     0   64M   0% /dev
tmpfs            13G     0   13G   0% /sys/fs/cgroup
shm              13G  4.0K   13G   1% /dev/shm
/dev/sda1       154G   41G  113G  27% /opt/bin
tmpfs            13G   36K   13G   1% /var/colab
tmpfs            13G     0   13G   0% /proc/acpi
tmpfs            13G     0   13G   0% /proc/scsi
tmpfs            13G     0   13G   0% /sys/firmware
drive           148G   45G  103G  31% /content/drive


In [ ]:
# Check CPU specs
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [42]:
dataset = datasets.load_dataset("json", data_files=data_fname, split=datasets.splits.Split("train"))

Using custom data configuration default-e65431e2a368a8d5
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-e65431e2a368a8d5/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02)


In [ ]:
def load_text(examples):
    return {"text": [item[0] + "\n" + item[1] for item in zip(examples['head_matter'], examples['opinion_text'])]}

exclude_columns = ["jurisdiction_id","court_id","decision_date", "head_matter","opinion_text","citation_ids"]
dataset=dataset.map(load_text, batched=True, num_proc=4, remove_columns=exclude_columns)

Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-e65431e2a368a8d5/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02/cache-fe528db6f38cdef8.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-e65431e2a368a8d5/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02/cache-83b05632ba7dd1c6.arrow


Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-e65431e2a368a8d5/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02/cache-c9f7b3972230c2b3.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-e65431e2a368a8d5/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02/cache-ace623e9e3282fb4.arrow


In [ ]:
def load_embeddings(examples):
    tokenized = tokenizer(examples["text"],
                          return_tensors="tf",
                          padding=True,
                          truncation=True,
                          max_length=512
                         )
    
    return {'embeddings': model(**tokenized)[1].numpy()}   # pulled out embeddings

In [22]:
dataset=dataset.map(load_embeddings, batched=True, batch_size=128)
dataset.save_to_disk("/content/drive/Shareddrives/CS109B-2021/final_project/data/subset/dataset_lbert_embeddings_pooled")

In [40]:
#! pip install faiss-gpu
! pip install faiss-cpu

In [32]:
import faiss

In [41]:
dataset.add_faiss_index(column='embeddings')
dataset.save_faiss_index('embeddings', '/content/drive/Shareddrives/CS109B-2021/final_project/data/subset/dataset_lbert_embeddings_pooled/embeddings.faiss')

ImportError: ignored

In [ ]:
! pip install faiss-cpu

     |████████████████████████████████| 8.2MB 11.2MB/s 


In [34]:
dataset = datasets.Dataset.load_from_disk("/content/drive/Shareddrives/CS109B-2021/final_project/data/subset/dataset_lbert_embeddings_pooled")

In [35]:
train_fname = "/content/drive/Shareddrives/CS109B-2021/final_project/data/subset/train_map.csv"
val_fname = "/content/drive/Shareddrives/CS109B-2021/final_project/data/subset/val_map.csv"
test_fname = "/content/drive/Shareddrives/CS109B-2021/final_project/data/subset/test_map.csv"


def fix_nulls_and_types(fname):
    df = pd.read_csv(fname)
    df = df.dropna()
    df = df.astype(int)
    df.to_csv(fname, index=False, index_label=False)
    return df
# fix_nulls_and_types(train_fname)
# fix_nulls_and_types(val_fname)
# fix_nulls_and_types(test_fname)

In [36]:
data_files={
    "train": train_fname, 
    "validation": val_fname, 
    "test": test_fname,
}
legalbert_dataset = datasets.load_dataset("csv", data_files=data_files)

Using custom data configuration default-76b2c87db56cbb6b


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-76b2c87db56cbb6b/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [ ]:
def load_legalbert_embeddings(examples):
    return {
        "case_embedding": dataset[examples["id"]]["embeddings"],
        "citation_embedding": dataset[examples["citation"]]["embeddings"],
    }

In [ ]:
legal_dataset = legalbert_dataset.map(load_legalbert_embeddings, batched=True, num_proc=4)
legal_dataset.save_to_disk("/content/drive/Shareddrives/CS109B-2021/final_project/data/subset/legalbert_pooled_dataset")

TypeError: ignored

In [ ]:
def generator_from_dataset(dataset):
    def _gen():
        for item in dataset:
            features = (item["case_embedding"], item["citation_embedding"])
            yield features, item['label']
    return _gen

def tf_dataset_from_dataset(dataset):
    dataset_generator = generator_from_dataset(dataset)
    tfdataset = tf.data.Dataset.from_generator(
        dataset_generator,
        output_signature=(
         (tf.TensorSpec(shape=(768,), dtype=tf.float32),
          tf.TensorSpec(shape=(768,), dtype=tf.float32),
         ),
         tf.TensorSpec(shape=(None), dtype=tf.int32))
    )
    tfdataset = tfdataset.apply(tf.data.experimental.assert_cardinality(len(dataset)))
    return tfdataset


In [ ]:
def tf_dataset_from_datasetv2(dataset):
    features = (dataset['case_embedding'],
                dataset['citation_embedding'])
    labels = dataset['label']
    return tf.data.Dataset.from_tensor_slices((features, labels))

In [ ]:
clf_dataset = datasets.DatasetDict.load_from_disk("/content/drive/Shareddrives/CS109B-2021/final_project/data/subset/legalbert_pooled_dataset", keep_in_memory=True)
clf_dataset.set_format(type='tensorflow', columns=['case_embedding', 'citation_embedding', 'label'])

train_dataset = clf_dataset['train']
val_dataset = clf_dataset['validation']
test_dataset = clf_dataset['test']

In [ ]:
train_dataset = tf_dataset_from_dataset(train_dataset)
val_dataset = tf_dataset_from_dataset(val_dataset)
test_dataset = tf_dataset_from_dataset(test_dataset)

In [ ]:
case_input = tf.keras.layers.Input(shape=(768,), dtype=tf.float32, name="case_input")
citation_input = tf.keras.layers.Input(shape=(768,), dtype=tf.float32, name="citation_input")

shared_stack = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dropout(0.3,),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.3,),
    tf.keras.layers.Dense(64, activation="relu"),
], name="shared_stack")

case_representation = shared_stack(case_input)
citation_representation = shared_stack(citation_input)
concatenated = tf.keras.layers.Concatenate()([case_representation, citation_representation])
concatenated = tf.keras.layers.Dropout(0.3,)(concatenated)
output = tf.keras.layers.Dense(1, activation="sigmoid")(concatenated)
clf_model = tf.keras.models.Model(inputs=[case_input, citation_input], outputs=[output], name="legalbert_model")

In [ ]:
clf_model.summary()

Model: "clf_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
case_input (InputLayer)         [(None, 768)]        0                                            
__________________________________________________________________________________________________
citation_input (InputLayer)     [(None, 768)]        0                                            
__________________________________________________________________________________________________
shared_stack (Sequential)       (None, 64)           566208      case_input[0][0]                 
                                                                 citation_input[0][0]             
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 128)          0           shared_stack[0][0]       

In [ ]:
legalbert_model.compile(loss="bce", metrics=["binary_accuracy"],  optimizer="adam")

In [ ]:
batch_size = 1024

train_dataset = train_dataset.repeat().shuffle(batch_size*4).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.repeat().batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.repeat().batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
             tf.keras.callbacks.ModelCheckpoint("../models/lbert_pooled_matching_v1", save_best_only=True,),
             tf.keras.callbacks.CSVLogger("../models/logs/lbert_pooled_matching_v1.csv"),
             tf.keras.callbacks.ReduceLROnPlateau(patience=3, verbose=1)
            ]
             

In [ ]:
history = clf_model.fit(
    train_dataset,
    steps_per_epoch=250,
    validation_data=val_dataset,
    validation_steps=100,
    verbose=1,
    epochs=15,
    callbacks=callbacks
)

Epoch 1/15
 15/250 [>.............................] - ETA: 21:05 - loss: 0.6827 - binary_accuracy: 0.5527